# Assignment 5 #
### Due: Friday, November 17th to be submitted via Canvas by 11:59 pm ###
### Total points: **65** ###

1. Homework Group - 68
2. Student Names - Anmol Agrawal (aa96674)
                   Aishwarya Parida (ap63595)

# Q1: Support Vector Machines (10 points)

In this question, we will explore support vector machines for the Spam Base dataset from the UCI repository.

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
seed = 42

In [3]:
!pip install ucimlrepo

In [4]:
from ucimlrepo import fetch_ucirepo
# fetch dataset
spambase = fetch_ucirepo(id=94)

# data (as pandas dataframes)
X = spambase.data.features
y = spambase.data.targets
print("Abstract:", spambase.metadata['abstract'])
print("Number of instances:", spambase.metadata['num_instances'])
print("Number of features:", spambase.metadata['num_features'])
print(spambase.variables['name'])

Abstract: Classifying Email as Spam or Non-Spam
Number of instances: 4601
Number of features: 57
0                 word_freq_make
1              word_freq_address
2                  word_freq_all
3                   word_freq_3d
4                  word_freq_our
5                 word_freq_over
6               word_freq_remove
7             word_freq_internet
8                word_freq_order
9                 word_freq_mail
10             word_freq_receive
11                word_freq_will
12              word_freq_people
13              word_freq_report
14           word_freq_addresses
15                word_freq_free
16            word_freq_business
17               word_freq_email
18                 word_freq_you
19              word_freq_credit
20                word_freq_your
21                word_freq_font
22                 word_freq_000
23               word_freq_money
24                  word_freq_hp
25                 word_freq_hpl
26              word_freq_george
27          

In [5]:
X.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [6]:
seed = 42
y = y.to_numpy().squeeze()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=seed)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

a. (5 points) Implement the following function to train SVMs with a specified kernel type, hyper-parameter search space, and random state on the Spam Base dataset. Do hyper-parameter search over $C$ using [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), setting the number of folds to 5. After finding the best C, please use it to train the final model and return both the final model and the best C.

In [7]:
def search_best_svm(kernel, C_search_space, random_state):
    best_score = -np.inf
    best_C=None
    for C in C_search_space:
        # Initialize an SVM classifier with the specified kernel type, C value, and random state
        ### START CODE ###
        svm = SVC(kernel=kernel, C=C, random_state=random_state)
        ### END CODE ###

        # Evaluate accuracy scores using 5-fold cross-validation scores
        ### START CODE ###
        scores = cross_val_score(svm, X_train, y_train, cv=5)
        ### END CODE ###

        # Compute the average score and compare with the current best score to update the best C
        ### START CODE ###
        average_score = np.mean(scores)
        if average_score > best_score:
            best_score = average_score
            best_C = C
        ### END CODE ###
        #print(f"C: {C} Avg Cross Val Score: {np.round(score, 4)}")
        print(f"C: {C}, Avg Cross Val Score: {average_score:.4f}")

    #print(f"Best C: {best_C}")
    print(f"Best C: {best_C}, Best Score: {best_score:.4f}")

    # Initialize the model using the specified kernel type, best C, and random state;
    # and then fit the model using training set
    ### START CODE ###
    model = SVC(kernel=kernel, C=best_C, random_state=random_state)
    model.fit(X_train, y_train)
    ### END CODE ###
    return model, best_C

b. (3 points) Run the function you implemented above to train SVMs with the search space of $C$ being [$0.1, 1, 10, 100$], random state set to 42, with the following three popular kernels: (i) linear (ii) polynomial (iii) RBF (Gaussian). Evaluate your final models on the test set and report their accuracies.

In [13]:
C_search_space = [0.1, 1, 10, 100]
random_state = 42
best_models = {}
accuracies = {}

for kernel in ['linear', 'poly', 'rbf']:
    model, best_C = search_best_svm(kernel, C_search_space, random_state)
    
    # Store the best model
    best_models[kernel] = model
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store the accuracy
    accuracies[kernel] = accuracy
    
    print(f"The best C for the {kernel} kernel is {best_C} with an accuracy of {accuracy*100:.2f}% on the test set.\n")


C: 0.1, Avg Cross Val Score: 0.9214
C: 1, Avg Cross Val Score: 0.9264
C: 10, Avg Cross Val Score: 0.9283
C: 100, Avg Cross Val Score: 0.9268
Best C: 10, Best Score: 0.9283
The best C for the linear kernel is 10 with an accuracy of 92.83% on the test set.

C: 0.1, Avg Cross Val Score: 0.6895
C: 1, Avg Cross Val Score: 0.7594
C: 10, Avg Cross Val Score: 0.8428
C: 100, Avg Cross Val Score: 0.8957
Best C: 100, Best Score: 0.8957
The best C for the poly kernel is 100 with an accuracy of 91.25% on the test set.

C: 0.1, Avg Cross Val Score: 0.8960
C: 1, Avg Cross Val Score: 0.9243
C: 10, Avg Cross Val Score: 0.9243
C: 100, Avg Cross Val Score: 0.9123
Best C: 10, Best Score: 0.9243
The best C for the rbf kernel is 10 with an accuracy of 93.54% on the test set.



c. (2 points) Train a logistic regression model using the training set. Compare its performance with that of the SVMs trained above.

In [23]:
from sklearn.metrics import classification_report
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy from Logistic Regression: {accuracy*100:.2f}%')
print(f'Classification Report:\n{report}')


print(f"Comparing with SVMs:")
print(f"Linear SVM Test Accuracy : {accuracies['linear']*100:.2f}%")
print(f"Polynomial SVM Test Accuracy : {accuracies['poly']*100:.2f}%")
print(f"Radial Basis Function SVM Test Accuracy : {accuracies['rbf']*100:.2f}%")

Accuracy from Logistic Regression: 92.23%
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      1089
           1       0.93      0.88      0.90       752

    accuracy                           0.92      1841
   macro avg       0.92      0.92      0.92      1841
weighted avg       0.92      0.92      0.92      1841

Comparing with SVMs:
Linear SVM Test Accuracy : 92.83%
Polynomial SVM Test Accuracy : 91.25%
Radial Basis Function SVM Test Accuracy : 93.54%


In this scenario, we observe that Radial Basis Function performs with the best accuracy (93.54%).The RBF kernel is useful in SVM when dealing with non-linearly separable data which suggests that there may be a non linear distribution in this data.

# Question 2 : Ensemble Methods for Classification (25 pts)

In this question, we will compare the performances of different ensemble methods for classification problems: [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html), [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) classifiers.

We will look at the [GiveMeSomeCredit](https://www.kaggle.com/c/GiveMeSomeCredit) dataset for this question. The dataset is extremely large so for this question we will only consider a subset which has been provided along with the notebook for this assignment. The dataset has already been split into train and test sets.

The task is to predict the probability that someone will experience financial distress in the next two years.

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
uploaded = files.upload()

Saving hw5_data.csv to hw5_data.csv


In [24]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.1 MB/s eta 0:00:0000:010:01


In [36]:
import pandas as pd

data = pd.read_csv('hw5_data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.571373,66,0,0.430620,9274.0,10,0,1,0,0.0
1,0,0.233999,56,0,0.257380,5656.0,12,0,0,0,0.0
2,0,0.299270,33,0,0.114575,4747.0,8,0,0,0,3.0
3,0,0.032165,41,0,0.308326,8490.0,8,0,1,0,0.0
4,0,0.050591,36,0,0.862627,3333.0,8,0,2,0,0.0


In [37]:
from sklearn.model_selection import train_test_split
y = data['SeriousDlqin2yrs']
X = data.drop(['SeriousDlqin2yrs'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7)

print('train:',X_train.shape, y_train.shape)
print('test:',X_test.shape, y_test.shape)

train: (3750, 10) (3750,)
test: (1250, 10) (1250,)


In [38]:
import matplotlib.pyplot as plt
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from time import time
import xgboost
%matplotlib inline

In [28]:
columns_list = list(X.columns)

a. (2.5 pts) Fit a Decision Tree Classifier with random_state = 14 for this classification problem. Report the accuracy_score and roc_auc_score on the test set.

In [39]:
def fit_classifier(clf, X_train, y_train):
    # Fit the classifier on the training set
  ### START CODE ###
    clf.fit(X_train, y_train)
  ### END CODE ###
    return clf

In [40]:
def evaluate_classifier(clf, X_test, y_test):
  # Compute the accuracy_score, and roc_auc_score on the test set
  ### START CODE ###
  y_pred = clf.predict(X_test)
  y_pred_proba = clf.predict_proba(X_test)[:, 1]

  acc_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred_proba)
  ### END CODE ###
  print("Accuracy_score: {}, ROC_AUC_score: {}".format(acc_score, auc_score))

In [41]:
print("Decision Tree")
# Initialize your decision tree classifier
### START CODE ###
dt_clf = DecisionTreeClassifier(random_state=14)
### END CODE ###

# Fit the classifier on the training set
dt_clf = fit_classifier(dt_clf, X_train, y_train)
evaluate_classifier(dt_clf, X_test, y_test)

Decision Tree
Accuracy_score: 0.888, ROC_AUC_score: 0.5854582176218127


b. (2.5 pts) Create a [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) of 25 classifiers (i.e, n_estimators=25) with random_state=14. Please use Decision Tree Classifier with random_state=14 as the base classifier. Report accuracy_score and roc_auc_score on the test data for this emsemble classifier.

In [43]:
print("Bagging of Decesion Trees")
# Initialize your bagging classifier
### START CODE ###
bag_clf = BaggingClassifier(estimator=None,n_estimators=25, random_state=14)
### END CODE ###

bag_clf = fit_classifier(bag_clf, X_train, y_train)
evaluate_classifier(bag_clf, X_test, y_test)

Bagging of Decesion Trees
Accuracy_score: 0.9272, ROC_AUC_score: 0.7866043928300934


c. (5 pts) In this question, you will fit a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) model on the training data for this classification task.

1. First, please find the best parameters (including *n_estimators*, *max_features* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the optimal parameters obtained by GridSearch.
2. Fit a model using the best parameters, and report the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) on test data.

In [49]:
def grid_search_for_classifier(clf, param_grid, X_train, y_train):
  # Grid search
     grid_search = GridSearchCV(clf, param_grid=param_grid)

  # Conduct grid search using the training set (1 line of code only)
  ### START CODE ###
     grid_search.fit(X_train, y_train)
  ### END CODE ###
     print(grid_search.best_params_)

  # Set the best paramters for your clf (1 line of code only)
  ### START CODE ###
   #clf.set_params(**grid_search.best_params_)
     clf=grid_search.best_estimator_
  ### END CODE ###
     return clf

In [50]:
def train_and_evaluate_classifier(clf, X_train, y_train, X_test, y_test):
  t0 = time()
  # Fit your classifier on the training set
  ### START CODE ###
  clf.fit(X_train, y_train)
  ### END CODE ###
  print("training time", round(time()-t0, 3), "s")

  t0 = time()
  y_pred = clf.predict(X_test)
  print("predict time", round(time()-t0, 3), "s")

  print("Confusion matrix: ")
  # Print the confusion matrix computed from the test set (1 line of code only)
  ### START CODE ###
  print(confusion_matrix(y_test, y_pred))
  ### END CODE ###


  ### START CODE ###
  y_pred_proba = clf.predict_proba(X_test)[:, 1]
  acc_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred_proba)
  ### END CODE ###

  print("Accuracy: {}, AUC_ROC: {}".format(acc_score, auc_score))
  return clf

In [53]:
param_grid = {"n_estimators": [1, 10, 50, 100],
              "max_features": [1, 5, 10, "auto"],
              "criterion": ['gini','entropy'],
              "random_state": [17]}

# Initialize your random forest classifier
### START CODE ###
rf_clf = RandomForestClassifier()
### END CODE ###
rf_clf = grid_search_for_classifier(rf_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(rf_clf, X_train, y_train, X_test, y_test)

/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

{'criterion': 'entropy', 'max_features': 1, 'n_estimators': 100, 'random_state': 17}
training time 0.178 s
predict time 0.012 s
Confusion matrix: 
[[1160    5]
 [  81    4]]
Accuracy: 0.9312, AUC_ROC: 0.8378591264832114


RandomForestClassifier(criterion='entropy', max_features=1, random_state=17)

d. (10 pts) This time, let us use [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) for the same task. For AdaBoost and XGBoost, please respectively find the best parameters (including *n_estimators, learning_rate*); fit your model using the best parameters, and report the confusion matrix and roc_auc_score on test data.

In [54]:
param_grid = {"n_estimators": [10, 100],
          "learning_rate": [0.01, 0.1, 0.5],
          "random_state": [17]
          }

In [56]:
# Initialize your AdaBoost classifier
### START CODE ###
ab_clf = AdaBoostClassifier()
### END CODE ###
ab_clf = grid_search_for_classifier(ab_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(ab_clf, X_train, y_train, X_test, y_test)

{'learning_rate': 0.1, 'n_estimators': 100, 'random_state': 17}
training time 0.188 s
predict time 0.008 s
Confusion matrix: 
[[1153   12]
 [  72   13]]
Accuracy: 0.9328, AUC_ROC: 0.8390254986114618


AdaBoostClassifier(learning_rate=0.1, n_estimators=100, random_state=17)

In [58]:
# Initialize your XGBoost classifier
### START CODE ###
xgb_clf = xgboost.XGBClassifier()
### END CODE ###
xgb_clf = grid_search_for_classifier(xgb_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(xgb_clf, X_train, y_train, X_test, y_test)

{'learning_rate': 0.01, 'n_estimators': 100, 'random_state': 17}
training time 0.048 s
predict time 0.002 s
Confusion matrix: 
[[1163    2]
 [  84    1]]
Accuracy: 0.9312, AUC_ROC: 0.8267962635698057


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=17, ...)

f. (5 pts) Compare the performance of decision tree from part a) with the ensemble methods. Briefly explain which of the three ensemble methods performed better and why?

1. The accuracy from a single decision tree is around 88.8%.
2. The accuracy from bagging model is 92.72%.
3. The accuracy from randomforest model is 93.12%
4. The accuracy from Adaboost model is 93.28% and accuracy from XGboost model is 93.12%

Also, the AUC_ROC score of single decison tree is lesser than that of ensembles.

We observe that the accuracy and AUC ROC scores of a single decision tree is lesser than the ensembles which aligns with the concept. Ensembles perfrom better than signle decision trees because they combine the predictions of multiple trees, which reduces the likelihood of overfitting and increases the model's ability to capture complex patterns in the data.

Among the ensembles, we observe that Adaboost model outperforms Bagging and Randomforest models. AdaBoost places more weight on the training instances that were previously misclassified.This can lead to a higher accuracy.

# Q3: CatBoost (10 points)

In this question you will learn about a boosting algorithm known as **CatBoost**. Please go through the two videos specified below to get a better understanding of the CatBoost algorithm and answer the questions that follow.

[Part-1](https://www.youtube.com/watch?v=KXOTSkPL2X4&ab_channel=StatQuestwithJoshStarmer)
[Part - 2](https://www.youtube.com/watch?v=3Bg2XRFOTzg&t=242s&ab_channel=StatQuestwithJoshStarmer)



a. **(5 points)** Briefly explain Ordered Target Encoding. What challenge does it try to address?

b. **(5 points)** Briefly describe the main advantages and disadvantages of CatBoost as compared to XGBoost.

#### a)
#### Ordered Target Encoding
Ordered Target Encoding is a technique used in machine learning to encode categorical variables to numericals by ranking them in relation to the target variable.In simpler terms, CatBoost calculates a statistic for each category by updating the average of the target variable for that category, based on the data points that come before the current row in the dataset. This means that, for any given row, CatBoost considers the historical data for that category to determine its statistic.

Ordered Target encoding helps avoid information leakage from the target variable because it dynamically updates the mean based on the order of the data.

Ordered Target Encoding can also help prevent overfitting by encoding categorical variables to numerical values such that it reflects their relationship with the target variable and reduces the risk of introducing noise into the model.

#### b)
#### Advantages of CatBoost over XGBoost:

1. CatBoost can naturally handle categorical data without the need for encoding, making it easier to work with datasets that have categorical variables. In contrast, XGBoost requires preprocessing of categorical features such as one hot encoding.

2. CatBoost is designed to be less prone to overfitting compared to XGBoost.

3. CatBoost can efficiently utilize GPUs for training, which can significantly speed up the training process, especially for categorical datasets. XGBoost also supports GPU acceleration, but CatBoost's implementation is often easier to work with.

#### Disadvantages of CatBoost over XGBoost:

1. CatBoost is slower to train compared to XGBoost,especially for large datasets. XGBoost on the other hand is known for its training speed.

2. CatBoost is less flexibility in hyperparameter tuning compared to XGBoost. XGBoost offers a wider range of hyperparameters for fine-tuning and customization.

# Q4: Convolutional Neural Network (20 points)
In this question, we will continue our exercise on the SVHN classification task from the previous homework, but this time we will be using Convolutional Neural Networks.

In [59]:
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

/Users/anmolagrawal/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [61]:
transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614))
        ])

train_dataset = torchvision.datasets.SVHN(root='.', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.SVHN(root='.', split='test', transform=transform, download=True)

100%|███████████████████████| 182040794/182040794 [00:08<00:00, 22579997.17it/s]


100%|█████████████████████████| 64275384/64275384 [00:05<00:00, 12688898.16it/s]


In [62]:
train_num = int(len(train_dataset) * 0.8)
val_num = len(train_dataset) - train_num
# Randomly split the training dataset into training dataset and validation dataset
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_num, val_num])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

a. (10 points) Build a convolutional neural network with the following sequential configuration. If not specified, please use the default setting of torch.nn.Conv2d. The output of the convolution layers will be fed into a fully-connected MLP. Then train the model with Adam optimizer (lr=1e-3) for 10 epochs. You should be able to achieve test accuracy of over 85%.



> Layer 1
*   2d convolution (# input channel=3, # output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 1
*   2d max pooling (kernel size=2)

> Layer 2
*   2d convolution (# output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 2
*   2d max pooling (kernel size=2)

> Layer 3
*   2d convolution (# output channel=32, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 3
*   2d max pooling (kernel size=2)

References:

*   https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html






In [63]:
class CNN(nn.Module):
    def __init__(self, pool=True):
        super(CNN, self).__init__()
        self.pool = pool

        # Create convolutional layers
        ### START CODE ###
        self.layer1 = nn.Sequential(nn.Conv2d(3, 16, kernel_size=3, padding=1),nn.BatchNorm2d(16),nn.ReLU())

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.layer2 = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, padding=1),nn.BatchNorm2d(16),nn.ReLU())

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.layer3 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, padding=1),nn.BatchNorm2d(32),nn.ReLU())

        self.pool3 = nn.MaxPool2d(kernel_size=2)
        ### END CODE ###

        # Create fully connected layers (nn.Linear)
        if self.pool:
            self.mlp1 = nn.Linear(32*4*4, 50)
        else:
            self.mlp1 = nn.Linear(32*32*32, 50)

        self.mlp2 = nn.Linear(50, 50)
        self.mlp3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.layer1(x)
        if self.pool:
            x = self.pool1(x)

        x = self.layer2(x)
        if self.pool:
            x = self.pool2(x)

        x = self.layer3(x)
        if self.pool:
            x = self.pool3(x)
        x = x.reshape(x.shape[0], -1)

        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = self.mlp3(x)

        return x

In [64]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Use the optimizer to perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    return avg_loss

@torch.no_grad()
def eval(model, loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Get model's prediction
        pred = torch.argmax(out, dim=1)

        # Count number of correct predictions
        correct = accuracy_score(target, pred, normalize=False)

        total_correct += correct
        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    acc = total_correct / total_num
    return avg_loss, acc

In [65]:
model1 = CNN(pool=True)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model1, train_loader, optimizer)
    val_loss, val_acc = eval(model1, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model1 = model1
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

100%|███████████████████████████████████████████| 58/58 [00:02<00:00, 24.72it/s]


Epoch: 1 Train Loss: 1.2207654907499785 Val Loss: 0.6438848191423231 Val Acc: 0.8101965601965602


100%|███████████████████████████████████████████| 58/58 [00:02<00:00, 24.41it/s]


Epoch: 2 Train Loss: 0.561296421849334 Val Loss: 0.5030663549379348 Val Acc: 0.8526481026481026


100%|███████████████████████████████████████████| 58/58 [00:02<00:00, 24.52it/s]


...

Epoch: 8 Train Loss: 0.317665394251994 Val Loss: 0.3702919187887165 Val Acc: 0.8901173901173901


100%|███████████████████████████████████████████| 58/58 [00:02<00:00, 24.69it/s]


Epoch: 9 Train Loss: 0.30021996573980386 Val Loss: 0.3743383329348456 Val Acc: 0.8880698880698881


100%|███████████████████████████████████████████| 58/58 [00:02<00:00, 24.61it/s]

Epoch: 10 Train Loss: 0.2900898263688564 Val Loss: 0.3637780602345075 Val Acc: 0.8948266448266449


In [66]:
_, test_acc = eval(best_model1, test_loader)
print(f"Test accuracy: {np.round(test_acc, 3)}")

100%|█████████████████████████████████████████| 102/102 [00:04<00:00, 24.06it/s]

Test accuracy: 0.89


b. (5 points) Use torch-summary to print a summary of the model. The number of parameters should be less than the one of the MLP we trained in the previous homework. Why does it have less number of parameters but have higher accuracy?

Reference
*   https://pypi.org/project/torch-summary/


In [67]:
!pip install torch-summary

In [69]:
from torchsummary import summary

# Create an instance of the CNN model
model = CNN()

# Print the model summary
summary(model, (3, 32, 32))  # Assuming input size of (3, 32, 32)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-3                         [-1, 16, 32, 32]          --
├─MaxPool2d: 1-2                         [-1, 16, 16, 16]          --
├─Sequential: 1-3                        [-1, 16, 16, 16]          --
|    └─Conv2d: 2-4                       [-1, 16, 16, 16]          2,320
|    └─BatchNorm2d: 2-5                  [-1, 16, 16, 16]          32
|    └─ReLU: 2-6                         [-1, 16, 16, 16]          --
├─MaxPool2d: 1-4                         [-1, 16, 8, 8]            --
├─Sequential: 1-5                        [-1, 32, 8, 8]            --
|    └─Conv2d: 2-7                       [-1, 32, 8, 8]            4,640
|    └─BatchNorm2d: 2-8                  [-1, 32, 8, 8]            64
|    └─R

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─ReLU: 2-3                         [-1, 16, 32, 32]          --
├─MaxPool2d: 1-2                         [-1, 16, 16, 16]          --
├─Sequential: 1-3                        [-1, 16, 16, 16]          --
|    └─Conv2d: 2-4                       [-1, 16, 16, 16]          2,320
|    └─BatchNorm2d: 2-5                  [-1, 16, 16, 16]          32
|    └─ReLU: 2-6                         [-1, 16, 16, 16]          --
├─MaxPool2d: 1-4                         [-1, 16, 8, 8]            --
├─Sequential: 1-5                        [-1, 32, 8, 8]            --
|    └─Conv2d: 2-7                       [-1, 32, 8, 8]            4,640
|    └─BatchNorm2d: 2-8                  [-1, 32, 8, 8]            64
|    └─R

CNNs use parameter sharing and local connectivity, which significantly reduce the number of parameters while efficiently capturing local and hierarchical patterns in images. This makes them more suitable for handling the high-dimensional data typical of images.

Also, pooling layers in CNNs provide the network with spatial invariance to input translation, making them better at recognizing objects in images regardless of their position. MLPs lack this mechanism, making them less effective for tasks where object location can vary.

c. (5 points) Train another CNN with the pool option set to False. What are the differences in terms of accuracy or computation caused by disabling max pooling? What are the effects of pooling operations in CNNs? (This might take some time. Watch a TV show while you're waiting for the results..)

In [70]:
model2 = CNN(pool=False)
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model2, train_loader, optimizer)
    val_loss, val_acc = eval(model2, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model2 = model2
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

100%|███████████████████████████████████████████| 58/58 [00:04<00:00, 11.68it/s]


Epoch: 1 Train Loss: 1.1317163487961954 Val Loss: 0.5977354190436146 Val Acc: 0.8203658203658204


100%|███████████████████████████████████████████| 58/58 [00:04<00:00, 11.73it/s]


Epoch: 2 Train Loss: 0.4888308106739258 Val Loss: 0.4922866175287316 Val Acc: 0.8506006006006006


100%|███████████████████████████████████████████| 58/58 [00:04<00:00, 11.83it/s]


...

Epoch: 8 Train Loss: 0.22629386106353908 Val Loss: 0.4354146088679697 Val Acc: 0.8813131313131313


100%|███████████████████████████████████████████| 58/58 [00:04<00:00, 11.62it/s]


Epoch: 9 Train Loss: 0.20229596501044186 Val Loss: 0.4676882262002761 Val Acc: 0.8741468741468742


100%|███████████████████████████████████████████| 58/58 [00:04<00:00, 11.83it/s]

Epoch: 10 Train Loss: 0.18000401606251382 Val Loss: 0.4502070361543411 Val Acc: 0.8757848757848757


In [71]:
_, test_acc = eval(best_model2, test_loader)
print(test_acc)

100%|█████████████████████████████████████████| 102/102 [00:08<00:00, 11.50it/s]

0.8607867240319607


Disabling max pooling in CNNs increases computational load due to larger spatial dimensions being processed and can reduce accuracy by making the network more sensitive to the exact location of features.

The accuracies we obtained aligns with this concept. With max pooling, the test accuracy is 89% but without max pooling, the accuracy drops to around 86%. This indicates that enabling pooling helps the model to generalise better and perform better on the unseen data.

Effect of Pooling in CNNs:

1. Pooling reduce the spatial dimensions of the feature maps, which cuts down the computational requirements and the number of parameters in the network. This helps in reducing overfitting.

2. Pooling also makes CNN less sensitive to the exact location of features in the image, thus enhancing the model's ability to recognize patterns irrespective of their position.